First few cells are for creating the master dataset (i.e by merging all csv files collected) 


In [42]:
#imports

import pandas as pd
import glob
from tqdm import tqdm

In [43]:
#specifies all file names using glob liberary and prints their path

all_files = glob.glob('../csv_files/*.csv')
print(all_files)

['../csv_files/3508153662.csv', '../csv_files/2028054341.csv', '../csv_files/00004.csv', '../csv_files/3602107603.csv', '../csv_files/3726298824.csv', '../csv_files/1203654458.csv', '../csv_files/996403123.csv', '../csv_files/3815991990.csv', '../csv_files/7567441.csv', '../csv_files/00005.csv', '../csv_files/Dataset2.csv', '../csv_files/00001.csv', '../csv_files/3722881981.csv', '../csv_files/3449474271.csv', '../csv_files/00003.csv', '../csv_files/Dataset1.csv', '../csv_files/Sumedha1.csv', '../csv_files/864405729.csv', '../csv_files/3119164873.csv', '../csv_files/00002.csv', '../csv_files/1975969790.csv', '../csv_files/Sumedha2.csv', '../csv_files/422034998.csv', '../csv_files/3658686332.csv', '../csv_files/2519810956.csv']


In [44]:
#Reads all files and inserts them into a list. Also accounts for inconsistencies 
# in the saving of the dataset performed by the juniors during data collection.

file_dfs = []

for i,file in tqdm(enumerate(all_files)):
    try:
        df = pd.read_csv(file , sep = '|')
        
    except:
        df = pd.read_csv(file)
    file_dfs.append(df)

        

25it [00:00, 27.51it/s]


In [45]:
#Merge and print length of full dataset

df = pd.concat(file_dfs , axis = 0 , ignore_index=True)
print(len(df))

54372


The following cells are for processing and selecting the right records in terms of experience and job roles.
First few cells deal with removal of useless columns



In [46]:
df.head()

,Unnamed: 0,Job Applicants:,Openings:,PG :,Posted:,UG :,experience,job_desc,key_skills,position,salary,Doctorate :
0,0,59,1.0,Post Graduation Not Required,30+ days ago,Any Graduate,2 - 5 years,Job description\n Technical Skills\nChartered...,"['Assurance', 'Business transformation', 'Fixe...",PPM041 - AM-Internal Audit,Not Disclosed,NaN
1,1,39,1.0,Post Graduation Not Required,30+ days ago,B.Tech/B.E. in Computers,4 - 6 years,Job description\nRoles and Responsibility\nDel...,"['Hibernate', 'Core Java', 'Front End', 'Oracl...",Java Full Stack Developer,Not Disclosed,NaN
2,2,130,1.0,Post Graduation Not Required,30+ days ago,B.Tech/B.E. in Computers,5 - 10 years,Job description\n.\nRoles and Responsibility\n...,"['Computer science', 'Data analysis', 'Analyti...",Sr. Data Scientist,Not Disclosed,NaN
3,3,42,1.0,Post Graduation Not Required,30+ days ago,B.Tech/B.E. in Computers,10 - 15 years,Job description\n\n\nRole purpose\n\nAs a Prin...,"['Linux', 'Coding', 'Project management', 'Ana...",Principle Architect,Not Disclosed,NaN
4,4,102,1.0,Post Graduation Not Required,30+ days ago,B.Tech/B.E. in Computers,3 - 8 years,Job description\n 3-8 years experience in das...,"['tableau', 'Reporting Analyst', 'SQL database...",Industry Consulting Prin. Consultant,Not Disclosed,NaN


In [47]:
#Drop Columns 

df.drop(['Unnamed: 0' , 'Job Applicants:' , "Openings:" , 'Posted:', 'PG :', "UG :" , 'salary'] , axis = 1 , inplace= True)
df.head()


,experience,job_desc,key_skills,position,Doctorate :
0,2 - 5 years,Job description\n Technical Skills\nChartered...,"['Assurance', 'Business transformation', 'Fixe...",PPM041 - AM-Internal Audit,NaN
1,4 - 6 years,Job description\nRoles and Responsibility\nDel...,"['Hibernate', 'Core Java', 'Front End', 'Oracl...",Java Full Stack Developer,NaN
2,5 - 10 years,Job description\n.\nRoles and Responsibility\n...,"['Computer science', 'Data analysis', 'Analyti...",Sr. Data Scientist,NaN
3,10 - 15 years,Job description\n\n\nRole purpose\n\nAs a Prin...,"['Linux', 'Coding', 'Project management', 'Ana...",Principle Architect,NaN
4,3 - 8 years,Job description\n 3-8 years experience in das...,"['tableau', 'Reporting Analyst', 'SQL database...",Industry Consulting Prin. Consultant,NaN


In [48]:
#Select records that start with 0 or 1 in the experience columns
#This records thus pertain to entry level jobs that require little to no experience 

df = df[df.experience.str.startswith('0') | df.experience.str.startswith('1 ')]

In [49]:
len(df)

14504

In [50]:
#show distribution of experience requirements

df.experience.value_counts()

1 - 3 years     2057
0 - 2 years     1895
0 - 1 years     1828
1 - 6 years     1553
1 - 5 years     1362
1 - 4 years     1311
0 - 3 years     1300
0 - 5 years     1057
1 - 2 years     1013
0 - 4 years      574
0 years          361
0 - 6 years       79
0 - 8 years       21
0 - 10 years      15
1 - 7 years       15
0 - 7 years       13
1 years           10
1 - 8 years        7
1 - 10 years       6
1 - 11 years       5
0 - 15 years       4
0 - 9 years        3
0 - 22 years       3
0 - 12 years       3
1 - 14 years       2
0 - 14 years       2
1 - 12 years       2
1 - 15 years       1
0 - 13 years       1
1 - 9 years        1
Name: experience, dtype: int64

In [51]:
#Processes the position column of the dataset. Strips spaces. 
#Filters out jobs that have a very broad range of experience requirements
#Print out the distribution of job roles

df.position = df.position.str.strip(' ')
df = df[~df['experience'].isin(['1 - 11 years' , '1 - 10 years' , '1 - 14 years' , '1 - 7 years' , '1 - 6 years' ,'0 - 5 years', '0 - 8 years' '1 - 8 years' , '1 - 5 years'])]
pd.set_option('display.max_rows' , 100)
df.position = df.position.str.lower()
df.position.value_counts()

php developer                                                             160
it recruiter                                                              130
business development executive                                            108
software developer                                                        106
android developer                                                          92
executive - collections                                                    78
sales executive                                                            74
web designer                                                               65
java developer                                                             61
web developer                                                              57
us it recruiter                                                            56
content writer                                                             51
python developer                                                

In [37]:
#First line is used to select the job roles that we want to chose. We can alter this line to select job roles for different subjects (For example python)
#Filters out the dataset based on the roles we choose
roles = ["python developer",
"python developer - django/data structure",
"developer associate - python",
"data analyst - python/artificial intelligence",
"python django developer",
"python freshers",
"freshers opening - python developers required !!",
"full stack developer (python)",
"python developer (trainee)",
"python or django developer",
"python odoo developer",
"software developer - python",
"lead python developer with b1 visa",
"aws python developer",
"python developer (fresher)",
"application developer proficient in python",
"jr. python developer",
"test python developer",
"internship - python developer",
"software engineer - python",
"python developers",
"python developer fresher",
"python developer intern",
"python developer trainee",
"python developer, data science",
"python/django developer",
"python developer intern || saas || stipend- 20k/month",
"python developer - intern",
"internship - python/django developer",
"python trainer & developer ( night shift and work from home)",
"freshers frontend developer (python), 0-1 year, bangalore",
"qa intern - python automation",
"gui python automation - intern",
"trainee-freshers-python-python developer",
"python developer: bangalore",
"python developers-intern",
"python developer (dm - 3424)",
"python developer - django / api integration",
"python developer - django/ numpy/ postgresql",
"python developer (dm - 3406)",
"python developer-bangalore",
"python / django developer",
"python/web developer",
"python scripter",
"python developer - django/flask",
"python developer - minimum 1 year experience",
"python developer, django, data science, machine learning freshers",
"python developer (dm- 3396) - fresher",
"python developer / machine learning / deep learning / (0 to 3 yrs)",
"software testing - java or python",
"python developer (internship cum placement)",
"freshers/ junior developers- python & sql",
"python developer - django/numpy/postgresql",
"python trainee / data extraction engineer / web crawlers",
"junior python developer",
"opening for python/odoo developer",
"python fullstack developer",
"python/django backend developer (freshers) - bangalore",
"hiring fresher for jr.developer(python/php/web designer)",
"required fresher for python developer(knowledge of django is advantage",
"training and placement on testing, mainframes, java and python",
"training and placements on java,python for freshers",
"android developer (intern)full stack developer python/ django + react",
"training and placement on java, python, testing, mainframes",
"python developer - end to end application - django/mysql",
"data science engineer - python",
"core python/linux developer",
"placements on java, python, and testing",
"training and placement on python",
"software engineer python developer (fresher)",
"python developer - mahape, navi mumbai",
"software developer - c++ / python / java",
"python fresher",
"training and placements on java,python,testing and mainframes.",
"training & placement on python",
"software developer python",
"business intelligence engineer - r shiny/sql/python",
"python/machine learning developer",
"freshers jr python developer",
"python developer (pl email at vijay.r@aroha.co.in)",
"java programmer python web developer",
"python - bangalore/ chennai / madurai",
"urgent hiring for fresher - 6 months (trained in python, spark/sap bo)",
"training and placement on java, testing and python",
"automation tester - selenium with python/c#",
"python django fresher",
"freshers - front / backend development - html / python, bangalore",
"0 to 2 years exp - trainers / faculties for python, c, c++, java sql.",
"python upgrade",
"intern / associate engineer / python",
"python /django developer & java and j2ee programmer",
"java developer python web developer",
"mlops architect with python",
"product associate - python/cloud",
"opening for python developer trainee @ noida",
"software engineer intern - python",
"cae engineer (catia, solidworks, ai & python)",
"python development internship",
"python engineer",
"testing java or python pune sus goan office job",
"python / odoo fresher",
"python/odoo trainee",
"odoo python intern",
"php python intern",
"online python tutor",
"python intern",
"internship in python",
"python developer freshers",
"python, tensorflow and keras",
"python research programmer distinguished founders",
"react js/ react native/ python developers (trained freshers & 1yr exp)",
"developer reactjs python",
"python, rest api software developer",
"trainer for java, c++, python, .net, machine learn and other languages",
"python developer/ backend engineer - remote till covid",
"fresher's programmers - java , .net, python, r programming",
"training and 100% placement on testing,python,java",
"full stack developer- python / react js",
"full stack developer-python",
"python/django web developer",
"python full stack developer",
"python developer (2873)",
"frappe/python developer",
"women in analytics, python/r - ta-996",
"backend developer - python",
"python trainer",
"python developer (job id: fld- 021)",
"senior python developer",
"python, c and dotnetcore programmer",
"java developer php html mysql python & web angular developer",
"python, testing, java , .net :fresher - urgent requirement",
"urgent hiring for python developer",
"trainee python developer",
"deep python with text analytics/clickstream analytics - l8",
"python (flask) + .net mvc developer",
"urgent requirement on java,python,testing freshers",
"junior developer python"]
df = df[df['position'].isin(roles)]
df.head()


,experience,job_desc,key_skills,position,Doctorate :
817,1 - 3 years,Job description\nPrimary Responsibilities\nLea...,"['Architecture', 'Data management', 'Analytica...","python developer, data science",NaN
2853,1 - 3 years,Job description\n1 3 years experience\n\nExper...,"['Engineering', 'Six Sigma', 'PMP', 'Business ...",python developer,NaN
3024,1 - 3 years,Job description\nPrimary Responsibilities\nLea...,"['Architecture', 'Data management', 'Analytica...","python developer, data science",NaN
4293,1 - 4 years,"Job description\nProduce clean, efficient code...","['Web application development', 'Backend', 'ng...",backend developer - python,NaN
5898,1 - 4 years,Job description\nRoles and Responsibilities :\...,"['Testing Tools', 'Application Software', 'CSS...",frappe/python developer,NaN


In [38]:
#Distribution of the final chosen job roles

df.position.value_counts()


python developer                                                          51
python developer - django/data structure                                  26
developer associate - python                                              25
data analyst - python/artificial intelligence                              4
python freshers                                                            4
python django developer                                                    4
full stack developer (python)                                              3
python developer (trainee)                                                 3
python or django developer                                                 3
python odoo developer                                                      3
freshers opening - python developers required !!                           3
python developer fresher                                                   2
python developer intern                                                    2

In [39]:
#distribution of experience requirements of chosen records
df.experience.value_counts()

0 - 1 years     100
0 - 2 years      53
0 - 3 years      49
0 - 4 years      20
0 years          20
1 - 3 years      10
1 - 2 years       5
0 - 6 years       5
1 - 4 years       4
0 - 8 years       2
0 - 13 years      1
0 - 10 years      1
Name: experience, dtype: int64

In [40]:
#Save the dataset 
#Change the name for different subjects. 

df.to_excel('../dataset/Excel/python_dataset.xlsx' , engine  = 'xlsxwriter' , index = False)

In [41]:
len(df)

270